### 1. Введение

#### 1.1. Организационная информация


Основная коммуникация: Telegram `DL - СПбГУ - 2023`. Если вас там нет, напишите `@snikolenko`, чтобы он вас туда добавил.

Материалы и домашние работы: `emkn`, GitHub: https://github.com/norsage/dl-mkn.git

Аттестация: по результатам результатов решений нескольких прикладных задач, где потребуется побить бейзлайн

Упражнения после занятия: опциональны, но очень полезны для развития интуиции


#### 1.2. План на сегодня: знакомство с PyTorch

1. Тензоры и операции над ними
2. Граф вычислений и автоматическое дифференцирование
3. Реализация новых операций с помощью `torch.autograd.Function`

#### 1.3. Подготовка окружения

1) Локально с помощью conda
   
   (можно установить по инструкции https://docs.conda.io/projects/miniconda/en/latest/index.html#quick-command-line-install)
   ```bash
   # создаём окружение dl-course
   conda create -n dl-course -y python=3.10
   # активируем окружение
   conda activate dl-course
   # устанавливаем Jupyter
   conda install jupyter notebook -c conda-forge

   # Устанавливаем pytorch (https://pytorch.org/get-started/locally/)
   # Пример для Linux / Windows с поддержкой GPU:
   conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
   # Для MacOS с ускорением MPS:
   # conda install pytorch::pytorch torchvision torchaudio -c pytorch
   ```

   Запустить ноутбук можно как в браузере:
   
   `jupyter notebook 01_pytorch_intro.ipynb`

   так и в VSCode, просто открыв файл и выбрав кернел для исполнения в правом верхнем углу (потребуется поставить расширения `Python` и `Jupyter`)
2) Используем Сolab https://colab.research.google.com/

### 2. PyTorch basics

In [ ]:
import torch
print(torch.__version__)
print(f"CUDA available: {torch.cuda.is_available()}")

#### 2.1. Тензоры, способы создания и атрибуты

https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html

Создать тензор можно многими способами:
1. Напрямую из объектов
2. Из массивов `numpy`
3. Из других тензоров
4. С константными и случайными значениями
5. Используя специальные функции для особых случаев

Основные атрибуты: ранг (`dim`), размерности (`shape`), тип значений (`type`), место размещения (`device`)

#### 2.2. Простые операции, функции, линейная алгебра

https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

Различных операций сотни, можно выделить несколько групп по назначению:
- создание (`linspace`, `arange`, `full`)
- индексация, срезы, объединения (`argwhere`, `cat`, `stack`, `tile`, `gather`)
- генерация случайных величин (`normal`, `bernoulli`, `randperm`)
- сериализация / десериализация (`save`, `load`)
- математические операции (`log`, `deg2rad`, `clamp`, `sigmoid`)
- агрегирование (`sum`, `mean`, `argmax`, `quantile`)
- линейная алгебра (`torch.linalg.*`)
- FFT (`torch.fft.*`)
- обработка сигналов (`torch.signal.*`)
- нейронные сети (`torch.nn.*`)
- ...

Многие операции также реализованы как методы класса `Tensor`

#### 2.3. Broadcasting

https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html#in-brief-tensor-broadcasting

Некоторые операции поддерживают `broadcast`, то есть размерности аргументов автоматически расширяются до нужного размера без копирования данных

Общие правила, когда это работает:
1. Все тензоры не пустые
2. При сравнении размеров тензоров, начиная с последней:
   1. Размерности совпадают, или
   2. Одна из размерностей равна $1$, или
   3. Размерность отсутствует в одном из тензоров


Благодаря `broadcast` многие вещи получается описать лаконично.

In [ ]:
# x=torch.empty(5,7,3)
# y=torch.empty(5,7,3)

# x=torch.empty((0,))
# y=torch.empty(2,2)

# x=torch.empty(5,3,4,1)
# y=torch.empty(  3,1,1)

# x=torch.empty(5,2,4,1)
# y=torch.empty(  3,1,1)

### 3. Autograd в PyTorch

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

#### 3.1. Вычислительный граф и дифференцирование

`autograd` строит DAG (directed acyclic graph) из объектов `torch.autograd.Function`, листья - входные тензоры, корни - выходные тензоры. Проход по графу позволяет рассчитать градиенты по правилу производной сложной функции (chain rule).

Прямой проход:
- расчёт значения выходного тензора
- построение графа и сохранение нужных для обратного прохода данных для каждой операции

Обратный проход (вызов `.backward()` у корня графа):
- расчёт градиентов и их накопление в артибуте `.grad` каждого тензора
- распространение вычислений далее до листьев графа

Рассмотрим выражение $f(x, y) = x^2 + xy + (x + y)^2$ и построим его граф:

<img src="expression_graph.png" style="background:white" width="300"/>

Запишем выражение для $f(x, y)$, задав начальные условия $x = 2.0, y = 2.0$.

`grad_fn` означает, что `f` не просто отдельный тензор, а связан с вычислительным графом и соответствует операции `Add`

Запустим backprop и убедимся, что градиенты рассчитаны правильно:

$\frac{\partial f}{\partial x} = 2x + y + 2(x + y)$

$\frac{\partial f}{\partial y} = x + 2(x + y)$

О чём стоит помнить:
- Повторная попытка выполнить `backward()` приводит к ошибке, т.к. после первого вызова граф уничтожается для экономии ресурсов. Если мы хотим сохранить граф для повторного обратного прохода, делаем `f.backward(retain_graph=True)`
- Градиенты в узлах графа не обнуляются автоматически, и при следующем вызове `backward` новое значение прибавится к старому
- Если по какой-то причине мы не хотим считать градиенты, вычисление выражения можно обернуть в контекст ```with torch.no_grad():```

#### 3.2. Отключение расчёта градиентов

Несколько способов:
1. Изменить значение атрибута тензора `requires_grad` напрямую
2. Использовать `torch.no_grad()` (как менеджер контекста или как декоратор)
3. Получить копию тензора с помощью метода `.detach()`

#### 3.3. Пример: логистическая регрессия

$\hat{y} = \sigma(w^T x + b)$

$\sigma(t) = \frac{1}{1 + \exp(-t)}$

$\text{CE}(y, \hat{y}) = -y \cdot \log \hat{y} - (1 - y) \log (1 - \hat{y})$

In [ ]:
torch.manual_seed(42)
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" style="background:white" width="700"/>

In [ ]:
loss

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Напишем цикл для поиска параметров, минимизирующих функцию ошибки

In [ ]:
b.grad

#### 3.4. Класс `torch.autograd.Function`

Когда может пригодиться:
1. Добавление недифференцируемых вычислений
2. Добавление операций, реализованный вне PyTorch (NumPy, SciPy, etc.)
3. Более эффективное использование ресурсов (комбинирование операций, обёртка над реализацией на `C++`)

In [ ]:
class MulConstant(torch.autograd.Function):
    @staticmethod
    def forward(tensor, constant):
        return tensor * constant

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor, constant = inputs
        ctx.constant = constant

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.constant, None

In [ ]:
class Mul(torch.autograd.Function):
    @staticmethod
    def forward(tensor1, tensor2):
        return tensor1 * tensor2

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor1, tensor2 = inputs
        ctx.tensor1 = tensor1
        ctx.tensor2 = tensor2

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.tensor1, grad_output * ctx.tensor2

### 4. Упражнения

#### 4.1. Функция Power
Используя сложение и умножение, реализуйте возведение в целочисленную степень FloatTensor как функцию autograd (т.е. наследника `torch.autograd.Function`)

In [ ]:
from typing import Any

class Power(torch.autograd.Function):
    @staticmethod
    def forward(tensor, p):
        ...

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        ...

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        ...

In [ ]:
assert(torch.all(Power.apply(torch.tensor([1, 2, 3]), 0) == torch.tensor([1, 1, 1])))
assert(torch.all(Power.apply(torch.tensor([1, 2, 3]), 2) == torch.tensor([1, 4, 9])))

#### 4.2. Многочлен
Найдите корень (он один) заданного полинома (очень хорошего!) с точностью до пяти знаков после запятой:
1. Используя бинарный поиск https://en.wikipedia.org/wiki/Binary_search_algorithm
2. Используя метод Ньютона https://en.wikipedia.org/wiki/Newton%27s_method
   
   Задаётся начальное приближение вблизи предположительного корня, после чего строится касательная к графику исследуемой функции в точке приближения, для которой находится пересечение с осью абсцисс. Эта точка берётся в качестве следующего приближения. И так далее, пока не будет достигнута необходимая точность.
   
   (hint: для вычисления производных используйте метод `backward()`)
   
   $x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)}$

Сравните скорость методов с помощью `%%timeit`, т.е. оцените, какой из них найдёт ответ быстрее

In [ ]:
from typing import Callable
Polynomial = Callable[[torch.FloatTensor], torch.FloatTensor]

def poly(x: torch.FloatTensor) -> torch.FloatTensor:
    return x ** 7 + 5 * x ** 3 + 17 * x - 9

In [ ]:
def bin_search_find_zero(poly: Polynomial) -> torch.FloatTensor:
  """Функция для бинарного поиска"""
  ...
  return ...

In [ ]:
def newton_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для метода Ньютона"""

    # первое приближение (не забываем про то, что понадобится градиент!)
    x = ...

    # останавливаемся, если значение функции достаточно близко к нулю
    tol = 10 ** -5

    # значение 
    val = ...

    # цикл обновления
    while ...:  # когда останавливаемся?
        # получаем градиент, обновляем значение x, оцениваем f(x)
        # hint: нужны ли нам градиенты, когда мы обновляем x?
        # hint: не забываем про обнуление градиента с прошлых шагов
        ...
    
    return x

In [ ]:
x = newton_find_zero(poly)
print(x)
print(poly(x))


In [ ]:
%%timeit
x = newton_find_zero(poly)

### 6. Что почитать / посмотреть
1. [Backpropagation: анимированное изложение](https://developers-dot-devsite-v2-prod.appspot.com/machine-learning/crash-course/backprop-scroll)
2. [You should understand backprop](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b) от Andrej Karpathy
3. [Neural Networks: Zero to Hero](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)